In [124]:
import pycosat
import secrets
import math
import numpy as np

In [125]:
file = open("Scrum.dimacs")

In [126]:
lines = file.readlines()

In [127]:
names = []
for line in lines[:124]:
    names.append(line.split(' ')[2][:-1])

In [129]:
dimacs = lines[125:]

In [130]:
len(dimacs)

255

In [131]:
cnf = [[int(s)  for s in line.split(' ') if int(s) != 0] for line in dimacs]

In [132]:
solutions = []
i = 0
for sol in pycosat.itersolve(cnf):
    if i == 100000:
        break
    i += 1
    solutions.append(sol)
print(i)

100000


In [133]:
class Item:
    def __init__(self, item):
        self.r = -1
        self.d = -1
        self.theta = -1
        self.item = item
        self.features = sum(item)
        self.totalcost = secrets.randbelow(100)
        self.knowndefects = secrets.randbelow(100)
        self.featuresused = secrets.randbelow(self.features)

In [134]:
binary_solutions = [[1 if val > 0 else 0 for val in sol] for sol in solutions]

items = [Item(item) for item in binary_solutions]
max_features = -math.inf
min_features = math.inf
max_totalcost = -math.inf
min_totalcost = math.inf
max_known = -math.inf
min_known = math.inf
max_featuresused = -math.inf
min_featuresused = math.inf

for x in items:
    if x.features > max_features:
        max_features = x.features
    if x.features < min_features:
        min_features = x.features
    if x.totalcost > max_totalcost:
        max_totalcost = x.totalcost
    if x.totalcost < min_totalcost:
        min_totalcost = x.totalcost
    if x.knowndefects > max_known:
        max_known = x.knowndefects
    if x.knowndefects < min_known:
        min_known = x.knowndefects
    if x.featuresused > max_featuresused:
        max_featuresused = x.featuresused
    if x.featuresused < min_featuresused:
        min_featuresused = x.featuresused

In [11]:
total_group = 10
west = []
east = []
westItems = []
eastItems = []
rand = secrets.choice(items)
max_r = -math.inf
min_r = math.inf
for x in items:
    x.r = sum(x.item)
    x.d = sum([a_i - b_i for a_i, b_i in zip(x.item, rand.item)])
    if x.r > max_r:
        max_r = x.r
    if x.r < min_r:
        min_r = x.r
for x in items:
    x.r = (x.r - min_r)/(max_r - min_r)
R = set([r.r for r in items])
for k in R:
    g = [item for item in items if item.r == k]
    g.sort(key=lambda x: x.d, reverse=True)
    for i in range(len(g)):
        g[i].theta = (2*math.pi*(i+1))/len(g)
thk = max_r / total_group
for a in range(total_group):
    g = [i for i in items if (a*thk)<= i.r <= ((a+1)*thk)]
    print(a)
    g.sort(key=lambda x: x.theta)
    if len(g) > 0:
        east.append(g[0])
        west.append(g[len(g)-1])
        for i in g:
            if i.theta <= math.pi:
                eastItems.append(i)
            else:
                westItems.append(i)

0
1
2
3
4
5
6
7
8
9


In [135]:
def split_bin(items, total_group):
    west = []
    east = []
    westItems = []
    eastItems = []
    rand = secrets.choice(items)
    max_r = -math.inf
    min_r = math.inf
    for x in items:
        x.r = sum(x.item)
        x.d = sum([a_i - b_i for a_i, b_i in zip(x.item, rand.item)])
        if x.r > max_r:
            max_r = x.r
        if x.r < min_r:
            min_r = x.r
    for x in items:
        x.r = (x.r - min_r)/(max_r - min_r + 10**(-32))
    R = set([r.r for r in items])
    for k in R:
        g = [item for item in items if item.r == k]
        g.sort(key=lambda x: x.d, reverse=True)
        for i in range(len(g)):
            g[i].theta = (2*math.pi*(i+1))/len(g)
    thk = max_r / total_group
    for a in range(total_group):
        g = [i for i in items if (a*thk)<= i.r <= ((a+1)*thk)]
        print(a)
        g.sort(key=lambda x: x.theta)
        if len(g) > 0:
            east.append(g[0])
            west.append(g[len(g)-1])
            for i in g:
                if i.theta <= math.pi:
                    eastItems.append(i)
                else:
                    westItems.append(i)
    return west, east, westItems, eastItems

In [13]:
def better():
    what = 1
    return what

In [136]:
from itertools import count

class TreeNode:
    _ids = count(0)

    def __init__(self, east, west, east_node, west_node):
        self.id = next(self._ids)
        self.east = east
        self.west = west
        self.east_node = east_node
        self.west_node = west_node


In [137]:
def sway(items, enough):
    if len(items) < enough:
        return TreeNode(items, None, None, None)
    delta1, delta2 = [], []
    west, east, west_items, east_items = split_bin(items, 10)
    east_node = sway(east_items, enough)
    west_node = sway(west_items,enough)
    node = TreeNode(east, west, east_node, west_node)
    return node

In [138]:
tree = sway(items, 100)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9


In [139]:
difference(tree.west[0], tree.east[0])

11

In [140]:
def difference(west, east):
    w = np.array(west.item)
    e = np.array(east.item)
    res = np.logical_xor(w, e)
    res = np.logical_not(res)
    return (len(res) - np.sum(res))

In [142]:
def bfs(tree, treshold):
    # maintain a queue of paths
    queue = []
    # push the first path into the queue
    queue.append([tree])
    smallest_diff = 150
    while queue:
        # get the first path from the queue
        path = queue.pop(0)
        path_id = [x.id for x in path]
        # get the last node from the path
        #import pdb; pdb.set_trace()
        node = path[-1]
        if(node.west):
            diff = difference(node.west[0], node.east[0])
            print(diff)
            # path found
            if diff < smallest_diff:
                smallest_diff = diff
            #checks to find question.
            if diff <= treshold:
                return path_id, node
            # enumerate all adjacent nodes, construct a new path and push it into the queue
            neighbors = []
            if(node.west_node):
                neighbors.append(node.west_node)
            if(node.east_node):
                neighbors.append(node.east_node)
            for adjacent in neighbors:
                new_path = list(path)
                new_path.append(adjacent)
                queue.append(new_path)
            
        
    print("Couldnt find", smallest_diff, path_id)

In [143]:
bfs(tree, 10)

11
11
14
15
12
9


([2554, 1276, 1275], <__main__.TreeNode at 0x22ebae176d8>)

In [76]:
def prLevel(root): 
  
    if (not root): 
        return
  
    # queue to hold tree node with level 
    q = []  
  
    # let root node be at level 1 
    q.append([root, 1]) 
  
    p = [] 
  
    # Do level Order Traversal of tree 
    while (len(q)):  
        p = q[0] 
        q.pop(0) 
        print("Level of", p[0].id, "is", p[1]) 
        if (p[0].west_node): 
            q.append([p[0].west_node, p[1] + 1]) 
        if (p[0].east_node): 
            q.append([p[0].east_node, p[1] + 1 ]) 

In [144]:
prLevel(tree)

Level of 2554 is 1
Level of 2553 is 2
Level of 1276 is 2
Level of 2552 is 3
Level of 1913 is 3
Level of 1275 is 3
Level of 636 is 3
Level of 2551 is 4
Level of 2232 is 4
Level of 1912 is 4
Level of 1593 is 4
Level of 1274 is 4
Level of 955 is 4
Level of 635 is 4
Level of 316 is 4
Level of 2550 is 5
Level of 2391 is 5
Level of 2231 is 5
Level of 2072 is 5
Level of 1911 is 5
Level of 1752 is 5
Level of 1592 is 5
Level of 1433 is 5
Level of 1273 is 5
Level of 1114 is 5
Level of 954 is 5
Level of 795 is 5
Level of 634 is 5
Level of 475 is 5
Level of 315 is 5
Level of 156 is 5
Level of 2549 is 6
Level of 2470 is 6
Level of 2390 is 6
Level of 2311 is 6
Level of 2230 is 6
Level of 2151 is 6
Level of 2071 is 6
Level of 1992 is 6
Level of 1910 is 6
Level of 1831 is 6
Level of 1751 is 6
Level of 1672 is 6
Level of 1591 is 6
Level of 1512 is 6
Level of 1432 is 6
Level of 1353 is 6
Level of 1272 is 6
Level of 1193 is 6
Level of 1113 is 6
Level of 1034 is 6
Level of 953 is 6
Level of 874 is 6
Level

Level of 1165 is 10
Level of 1161 is 10
Level of 1156 is 10
Level of 1150 is 10
Level of 1145 is 10
Level of 1141 is 10
Level of 1136 is 10
Level of 1131 is 10
Level of 1126 is 10
Level of 1122 is 10
Level of 1117 is 10
Level of 1109 is 10
Level of 1104 is 10
Level of 1100 is 10
Level of 1095 is 10
Level of 1090 is 10
Level of 1085 is 10
Level of 1081 is 10
Level of 1076 is 10
Level of 1070 is 10
Level of 1065 is 10
Level of 1061 is 10
Level of 1056 is 10
Level of 1051 is 10
Level of 1046 is 10
Level of 1042 is 10
Level of 1037 is 10
Level of 1030 is 10
Level of 1025 is 10
Level of 1021 is 10
Level of 1016 is 10
Level of 1011 is 10
Level of 1006 is 10
Level of 1002 is 10
Level of 997 is 10
Level of 991 is 10
Level of 986 is 10
Level of 982 is 10
Level of 977 is 10
Level of 972 is 10
Level of 967 is 10
Level of 963 is 10
Level of 958 is 10
Level of 949 is 10
Level of 944 is 10
Level of 940 is 10
Level of 935 is 10
Level of 930 is 10
Level of 925 is 10
Level of 921 is 10
Level of 916 is 

Level of 948 is 11
Level of 945 is 11
Level of 943 is 11
Level of 942 is 11
Level of 939 is 11
Level of 936 is 11
Level of 934 is 11
Level of 933 is 11
Level of 929 is 11
Level of 926 is 11
Level of 924 is 11
Level of 923 is 11
Level of 920 is 11
Level of 917 is 11
Level of 915 is 11
Level of 914 is 11
Level of 909 is 11
Level of 906 is 11
Level of 904 is 11
Level of 903 is 11
Level of 900 is 11
Level of 897 is 11
Level of 895 is 11
Level of 894 is 11
Level of 890 is 11
Level of 887 is 11
Level of 885 is 11
Level of 884 is 11
Level of 881 is 11
Level of 878 is 11
Level of 876 is 11
Level of 875 is 11
Level of 869 is 11
Level of 866 is 11
Level of 864 is 11
Level of 863 is 11
Level of 860 is 11
Level of 857 is 11
Level of 855 is 11
Level of 854 is 11
Level of 850 is 11
Level of 847 is 11
Level of 845 is 11
Level of 844 is 11
Level of 841 is 11
Level of 838 is 11
Level of 836 is 11
Level of 835 is 11
Level of 830 is 11
Level of 827 is 11
Level of 825 is 11
Level of 824 is 11
Level of 821

Level of 90 is 12
Level of 82 is 12
Level of 81 is 12
Level of 70 is 12
Level of 69 is 12
Level of 61 is 12
Level of 60 is 12
Level of 51 is 12
Level of 50 is 12
Level of 42 is 12
Level of 41 is 12
Level of 31 is 12
Level of 30 is 12
Level of 22 is 12
Level of 21 is 12
Level of 12 is 12
Level of 11 is 12


In [145]:
def rankFeatures(items):
    count = np.zeros(len(items[0].item))
    for item in items:
        count = np.add(count, item.item)
    rank = np.zeros(len(count))
    for i, v in enumerate(count):
        if v == 0:
            rank[i] = -1
            print("No", names[i])
        if v == (len(items)):
            rank[i] = -1
            print("All", names[i])
    return count, rank

In [146]:
len(items)

100000

In [147]:
count, rank = rankFeatures(items)

No Experienced_professionals
No r_5_minutes
All Presence_d
All r_8_hours
No PO_alone
No Two_half_weeks
No Google_docs
All PB_ID
All Physical
No SM_DevTeam
No Daily
No Three_weeks
All Status
No Story_points_and_points_of_value
No From_6_to_8
No Google_Group
No Apo6
No Apo5
All Roles
No Apo4
No Apo3
No Apo2
No Two
No Apo1
All Artifacts
No Many_times_a_day
No t_15_to_35_minutes
All Estimated_Time
All Sprint
All Planning
All Blocked
No RP
All Estimate
All Selection
All One
No Twice_a_week
No Equal_9
All SB
No More_two
All DevTeam2
All Events
No Less_3
No Chat
All SM
All Progress
All Num_persons
All Rules
All lenght
No Planning_Poker
All Composing_dt
All Frequency
No One_week
No PO_DevTeam
No Online
No User_Stories
No Email
No three_hours
All Three_times_week
No one_week_p
All PO_DevTeam2
No Atd12
No One_week_1
All In_progress
No Atd10
No Atd11
No more_9
All Representation
All Composing
No LOEs
All physical_dt
All Presence_dt
All Num_person_dt
All Time_d
No Cpo10
All Prioritization
No Time


In [150]:
for i, r in enumerate(rank):
    if r == 0:
        print(names[i])
print(len(rank))

Cpo7
Cpo8
Cpo9
Eight_weeks
t_5_minutes
Two_weeks
Ppo1
Ppo2
Apo11
Apo13
Apo14
t_15_minutes
t_15_to_20_minutes
PO_SM_DevTeam
Atd8
Atd7
Atd6
Refinery_Requirements
Atd5
Atd3
Atd2
124


In [ ]:
def itemDom(west, east):
    n = 4
    s1 = 0.0
    s2 = 0.0
    westfeatures   = (west.features - min_features)
    y   = i.rows[r2][c]
    x   = NumNorm(i.cols[c], x)
    y   = NumNorm(i.cols[c], y)
    s1 -= 2.72 ^ ( i.cols[c].w * (x - y)/n )
    s2 -= 2.72 ^ ( i.cols[c].w * (y - x)/n )
  
  return s1/n < s2/n
